### script to apply SSK BO over an space with context free grammar constraints
### we demonstrate on symbolic regression task

In [1]:
import numpy as np
from numpy import *
import emukit
import re
import matplotlib.pyplot as plt
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace
from emukit.core.optimization import RandomSearchAcquisitionOptimizer
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.core.loop import FixedIterationsStoppingCondition
import warnings
warnings.filterwarnings('ignore')

In [3]:
#import our code
from boss.code.CFG.CFG import Grammar
from boss.code.parameters.cfg_parameter import CFGParameter
from boss.code.parameters.cfg_parameter import unparse
from boss.code.optimizers.GrammarGeneticAlgorithmAcquisitionOptimizer import GrammarGeneticProgrammingOptimizer
from boss.code.emukit_models.emukit_ssk_model import SSK_model

# Define problem (objective and space)

In [4]:
# define grammar rules

	# define our arithmetic grammar
# note that we have to map ( -> lb and ) -> rb
# as the tree kernel reads these sybmols to seperate branches

# also require 'dummy' productions for arithmetic expressions
# as our parser in the kernel requires all terminal nodes to be connected to a single node
# e.g.
#         S
#      /  |  \
#     S  ADD  T
#     |   |   |
#     T  "a" "1"
#     |
#   "x"   
# is the string "x + 1"
# and is represented as '(S (S (T x)) (ADD a) (T 1))'

grammar = Grammar.fromstring("""
 S -> S ADD T | S TIMES T | S DIVIDE T | T
 T -> LB S RB | SIN S RB | EXP S RB
 ADD -> "+"
 TIMES -> "*"
 DIVIDE -> "/"
 LB -> "lb"
 RB -> "rb"
 SIN -> "sin"
 EXP -> "exp"
 T -> "x" | "1" | "2" | "3"
 """)

In [5]:
# True expression
# This is the target expression that we wish to learn
true = '1/3+x+sin(x*x)'
x = np.linspace(-10,10,1000)
y = np.array(eval(true)) 
# Objective function
# we wish to find symbolic expressions X that are close in MSE error between X(x) and true y
#
# Following Grammar VAE paper we put in a hard limit on worse MSE 
# and optimize log(1+MSE), as exponential terms in expressions can give
# large MSE, which are hard to model with a GP

def objective(X):
    #X needs to come in as a 2d numpy array in raw form '3*x+2' 
    X=np.atleast_2d(X)
    f_evals=np.zeros((X.shape[0],1))
    for i in range(X.shape[0]):
        # format correctly for numpy
        string = X[i][0]
        string = string.replace(" ","")
        string = string.replace("lb","(")
        string = string.replace("rb",")")
        string = string.replace("exp","exp(")
        string = string.replace("sin","sin(")
        # hard limit of 1000
        result = np.log(1+np.minimum(np.mean((np.array(eval(string)) - y)**2), 1000))
        if np.isnan(result):
        	result = np.log(1+1000)
        f_evals[i] = result
    # return log(1+MSE) of each input X
    return f_evals

# define search space (length refers to number of terminals in strings)
length=20
min_length=3
space = ParameterSpace([CFGParameter("grammar",grammar,max_length=length,min_length=min_length)])

# Collect initial points

In [6]:
# collect initial design (uniform sample)
np.random.seed(123)
random_design = RandomDesign(space)
initial_points_count = 15
X_init = random_design.get_samples(initial_points_count)
X_init_strings = unparse(X_init)
Y_init = objective(X_init_strings)

# Explain Methods

In [ ]:
# we perform optimziation using our SSK-approach and random search
# VAE baselines are availible for Grammar VAEs and Character VAES at https://github.com/mkusner/grammarVAE

# 1) Perform BO with SSK

In [7]:
# build BO loop
# fit SSK model
# just a single restart when fitting kernel params for demo 
model = SSK_model(space,X_init_strings,Y_init,max_subsequence_length=5,n_restarts=3)
# Load core elements for Bayesian optimization
expected_improvement = ExpectedImprovement(model)
# either use genetic algorithm or random search to optimize acqusition function
optimizer =  GrammarGeneticProgrammingOptimizer(space,dynamic=True,population_size=100,tournament_prob=0.5,p_crossover= 0.8, p_mutation=0.1)
# optimizer = RandomSearchAcquisitionOptimizer(space,10000)
bayesopt_loop_SSK= BayesianOptimizationLoop(model = model, 
                                         space = space,
                                         acquisition = expected_improvement,
                                         acquisition_optimizer = optimizer)
# add loop summary
def summary(loop, loop_state):
    print("Performing BO step {}".format(loop.loop_state.iteration))
bayesopt_loop_SSK.iteration_end_event.append(summary)

reconstraining parameters GP_regression.sk.Gap_decay
reconstraining parameters GP_regression.sk.Match_decay


In [ ]:
# run BO loop for 25 steps 
np.random.seed(123)
stopping_condition = FixedIterationsStoppingCondition(i_max = 25) 
bayesopt_loop_SSK.run_loop(objective, stopping_condition)

Optimization restart 1/3, f = 22.407539932552716
Optimization restart 2/3, f = 22.407541088667664
Optimization restart 3/3, f = 22.407539246018814
Performing BO step 1
Optimization restart 1/3, f = 22.844085582090226
Optimization restart 2/3, f = 22.844090496401268
Optimization restart 3/3, f = 22.84408945258417
Performing BO step 2
Optimization restart 1/3, f = 25.45547072205934
Optimization restart 2/3, f = 25.45547270865916
Optimization restart 3/3, f = 25.45546834556673
Performing BO step 3
Optimization restart 1/3, f = 27.891916954918905
Optimization restart 2/3, f = 27.891918624426403
Optimization restart 3/3, f = 27.891918237075714
Performing BO step 4
Optimization restart 1/3, f = 28.36068739009751
Optimization restart 2/3, f = 28.36069096042931
Optimization restart 3/3, f = 28.360690144347096
Performing BO step 5
Optimization restart 1/3, f = 27.795573591637265
Optimization restart 2/3, f = 27.795584862375513
Optimization restart 3/3, f = 27.795588370002662
Performing BO step 

# Perform random search

In [ ]:
# also see performance of random search 
#(starting from the initialization used by the other approaches)
np.random.seed(123)
Y_random=np.vstack([Y_init,objective(unparse(random_design.get_samples(25)))])

# plot results

In [ ]:
# plot results 
# recall that first 15 points are a random sample shared by all the methods
plt.plot(np.minimum.accumulate(bayesopt_loop_SSK.loop_state.Y),label="SSk")
plt.plot(np.minimum.accumulate(Y_random),label="Random Search")
plt.yscale("log")
plt.ylabel('Current best score')
plt.xlabel('Iteration')
plt.legend()